In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from apiclient.discovery import build
import pandas as pd
import json
from json import JSONDecodeError
import geopandas as gpd
from shapely.geometry import box, Point

In [2]:
KEY_FILE = 'GeosciencePortal-eeac25f8efb3.json'
ACCOUNT_NAME = 'AUSGIN website'
PROPERTY_NAME = 'AUSGIN'
PROFILE_NAME = 'AUSGIN - exclude internal traffic'

def get_service(key_file,scopes):
  credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE, scopes = ['https://www.googleapis.com/auth/analytics.readonly'])

  service = build("analytics", "v3", credentials=credentials)
  return service


def get_profile_id(service):
  service.management().accounts().list().execute()
  accounts = service.management().accounts().list().execute()

  account = [acc.get('id') for acc in accounts.get('items') if acc.get('name') == ACCOUNT_NAME][0]

  properties = service.management().webproperties().list(accountId=account).execute()

  web_property = [prop.get('id') for prop in properties.get('items') if prop.get('name') == PROPERTY_NAME][0]

  profiles = service.management().profiles().list(accountId=account, webPropertyId=web_property).execute()
          
  return [prof.get('id') for prof in profiles.get('items') if prof.get('name') == PROFILE_NAME][0]


def get_results(service, profile):
  # Use the Analytics Service Object to query the Core Reporting API
  # for the number of sessions within the past seven days.
  return service.data().ga().get(
    ids='ga:' + profile,
    start_date='2016-10-01',
    end_date='today',
    metrics='ga:totalEvents',
    max_results=10000,
    dimensions='ga:eventCategory,ga:eventAction,ga:eventLabel').execute()



def print_results(results):
  if results:
    print('View (Profile): {0}'.format(results.get('profileInfo').get('profileName')))
    print('Total Events: {0}'.format(results.get('rows')[0][0]))

  else:
    print('No results found')

In [3]:
service = get_service(KEY_FILE, ['https://www.googleapis.com/auth/analytics.readonly'])
profile = get_profile_id(service)
print_results(get_results(service, profile))

View (Profile): AUSGIN - exclude internal traffic
Total Events: Add:CSWRecord


In [4]:
def label_filterer(data):
  if data.startswith("Filter:"):
    string = data.replace("Filter:","")
    return json_parser(string)
  elif data.startswith("parameters:"):
    string = data.replace("parameters:","")
    return json_parser(string)
  else:
    return None
  


In [5]:
def json_parser(json_string):
  try: 
    parsed = json.loads(json_string)
  except (JSONDecodeError, TypeError):
    parsed = "error in parsing - please check"
  return parsed

In [6]:
results = get_results(service, profile)

In [7]:
results

{'columnHeaders': [{'columnType': 'DIMENSION',
   'dataType': 'STRING',
   'name': 'ga:eventCategory'},
  {'columnType': 'DIMENSION', 'dataType': 'STRING', 'name': 'ga:eventAction'},
  {'columnType': 'DIMENSION', 'dataType': 'STRING', 'name': 'ga:eventLabel'},
  {'columnType': 'METRIC', 'dataType': 'INTEGER', 'name': 'ga:totalEvents'}],
 'containsSampledData': False,
 'id': 'https://www.googleapis.com/analytics/v3/data/ga?ids=ga:124230702&dimensions=ga:eventCategory,ga:eventAction,ga:eventLabel&metrics=ga:totalEvents&start-date=2016-10-01&end-date=today&max-results=10000',
 'itemsPerPage': 10000,
 'kind': 'analytics#gaData',
 'profileInfo': {'accountId': '79593123',
  'internalWebPropertyId': '118718285',
  'profileId': '124230702',
  'profileName': 'AUSGIN - exclude internal traffic',
  'tableId': 'ga:124230702',
  'webPropertyId': 'UA-79593123-1'},
 'query': {'dimensions': 'ga:eventCategory,ga:eventAction,ga:eventLabel',
  'end-date': 'today',
  'ids': 'ga:124230702',
  'max-results'

In [8]:
df=pd.DataFrame(results.get('rows'), columns=[r.get('name') for r in results.get('columnHeaders')])

In [9]:
df.to_excel('raw_analytics.xlsx')

In [10]:
df['parsed'] = df["ga:eventLabel"].apply(label_filterer)

In [11]:
new_frame = df['parsed'].apply(pd.Series)

/Users/michael/miniconda3/lib/python3.5/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/michael/miniconda3/lib/python3.5/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  result = result.union(other)
/Users/michael/miniconda3/lib/python3.5/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: unorderable types: str() > int(), sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/michael/miniconda3/lib/python3.5/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: unorderable types: str() > int(), sort order is undefined for incomparable objects
  result = result.union(other)


In [12]:
parsed_json = pd.concat([df[:],new_frame[:]], axis=1)

In [13]:
bbox = json.loads(parsed_json['bbox'][0])

In [14]:
def parse_bbox(json):

  bbox = json_parser(json)

  if type(bbox) == dict:
      return box(*map(lambda x: float(x),(bbox['westBoundLongitude'],bbox['northBoundLatitude'],bbox['eastBoundLongitude'],bbox['southBoundLatitude'])))
  else:
    return None

In [15]:
df['shape'] =parsed_json["bbox"].apply(parse_bbox)

In [16]:
shapefile_frame = df[df['shape'].notnull()]

In [17]:
gdf = gpd.GeoDataFrame(shapefile_frame, geometry='shape')

In [18]:
gdf["centroid"] = gdf["shape"].centroid

In [19]:
map_centre=Point(133.3,-26)

In [20]:
gdf["shape"][4].distance(map_centre)

0.0

In [21]:
gdf["distance"] = gdf["shape"].centroid.distance(map_centre)
gdf["area"] = gdf["shape"].area

In [26]:
interesting_boxes = gdf[gdf["distance"]>0]
australia_bounds = box(110,-45,155,-10)
interesting_boxes = interesting_boxes[interesting_boxes["area"]<4]

In [27]:
interesting_boxes = interesting_boxes[interesting_boxes["shape"].centroid.intersects(australia_bounds)]

In [28]:
interesting_boxes.drop("centroid", inplace=True, axis=1)

In [29]:
interesting_boxes.to_file('bounding_boxes', driver='ESRI Shapefile')

In [30]:
interesting_boxes

,ga:eventCategory,ga:eventAction,ga:eventLabel,ga:totalEvents,parsed,shape,distance,area
1,Add:CSWRecord,Layer:2_5M_Dykes_2015,"Filter:{""opacity"":1,""bbox"":""{\""westBoundLongit...",1,"{'opacity': 1, 'bbox': '{""westBoundLongitude"":...","POLYGON ((154 -30, 154 -31, 151 -31, 151 -30, ...",19.720294,3.0000
35,Add:KMLRecord,Layer:Australie12876-2124.kml,"Filter:{""bbox"":""{\""westBoundLongitude\"":\""129\...",1,"{'bbox': '{""westBoundLongitude"":""129"",""southBo...","POLYGON ((130 -22, 130 -22.5, 129 -22.5, 129 -...",5.338773,0.5000
36,Add:KMLRecord,Layer:Australie12876-2124.kml,"Filter:{""bbox"":""{\""westBoundLongitude\"":\""129\...",1,"{'bbox': '{""westBoundLongitude"":""129"",""southBo...","POLYGON ((131 -22, 131 -23, 129 -23, 129 -22, ...",4.810405,2.0000
38,Add:KMLRecord,Layer:Australie12876-2374.png,"Filter:{""bbox"":""{\""westBoundLongitude\"":\""129\...",1,"{'bbox': '{""westBoundLongitude"":""129"",""southBo...","POLYGON ((130 -24.5, 130 -25.2, 129 -25.2, 129...",3.970202,0.7000
64,Add:KMLRecord,Layer:Gold Mine Way Points QLD(GARMIN).gpi,"Filter:{""bbox"":""{\""westBoundLongitude\"":\""144....",1,"{'bbox': '{""westBoundLongitude"":""144.3"",""south...","POLYGON ((144.3 -16, 144.3 -16, 144.3 -16, 144...",14.866069,0.0000
65,Add:KMLRecord,Layer:Lithium.kml,"Filter:{""bbox"":""{\""westBoundLongitude\"":\""118....",1,"{'bbox': '{""westBoundLongitude"":""118.8"",""south...","POLYGON ((119.1 -21, 119.1 -21.1, 118.8 -21.1,...",15.179756,0.0300
68,Add:KMLRecord,Layer:equipop_DReduction_integ05.kml,"Filter:{""bbox"":""{\""westBoundLongitude\"":\""132\...",1,"{'bbox': '{""westBoundLongitude"":""132"",""southBo...","POLYGON ((134 -22.7, 134 -23.6, 132 -23.6, 132...",2.865746,1.8000
72,Add:KnownLayer,Layer:ASTER Map AlOH group composition,"Filter:{""bbox"":""{\""westBoundLongitude\"":\""138\...",1,"{'bbox': '{""westBoundLongitude"":""138"",""southBo...","POLYGON ((140 -30.4, 140 -31.3, 138 -31.3, 138...",7.484150,1.8000
79,Add:KnownLayer,Layer:ASTER Map AlOH group composition,"Filter:{""opacity"":1,""bbox"":""{\""westBoundLongit...",1,"{'opacity': 1, 'bbox': '{""westBoundLongitude"":...","POLYGON ((124 -17, 124 -18, 121 -18, 121 -17, ...",13.743726,3.0000
81,Add:KnownLayer,Layer:ASTER Map AlOH group composition,"Filter:{""opacity"":1,""bbox"":""{\""westBoundLongit...",1,"{'opacity': 1, 'bbox': '{""westBoundLongitude"":...","POLYGON ((137.9 -33.5, 137.9 -33.6, 137.6 -33....",8.763846,0.0300
